# 📘 Data Cleaning Notebook — AquaSafe

**Notebook:** `02_data_cleaning.ipynb`

**Input:** `data/raw/NWMP_August2025_MPCB_0.csv`

**Output:** `data/processed/cleaned_water_quality.csv` (structurally clean, may contain NaN)

---

## 🎯 Objective

Perform **structural cleaning** on raw water quality data to prepare it for feature engineering.

### ✅ What This Notebook Does:
| Task | Description |
|------|-------------|
| Column Standardization | Normalize names to snake_case |
| Type Normalization | Parse BDL annotations, convert to numeric |
| Coordinate Conversion | DMS → Decimal Degrees |
| Target Mapping | Verbose labels → A/B/C/E codes |
| Column Removal | Drop leakage, metadata, identifier columns |
| Invalid Row Removal | Remove rows with missing/invalid target |

### ❌ What This Notebook Does NOT Do:
| Task | Reason | Where It Happens |
|------|--------|------------------|
| Imputation | Must happen AFTER train-test split to avoid leakage | Notebook 03 |
| Encoding | Must happen AFTER train-test split to avoid leakage | Notebook 03 |
| Scaling | Part of model pipeline | Notebook 04 |
| Train-Test Split | Belongs in feature engineering | Notebook 03 |

### 💡 Why This Approach?
Imputation and encoding on full data causes **data leakage** — test set statistics would contaminate training data, leading to overly optimistic performance estimates.

---
## 🔧 Section 1: Setup & Imports

In [1]:
# ============================================================================
# IMPORTS
# ============================================================================

import pandas as pd
import numpy as np
import os
from pathlib import Path

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Project modules
from utils.config import DATA_PATH, DATA_DIR
from src.data_preprocessing.create_dataframe import create_dataframe

# Display settings
pd.set_option('display.float_format', '{:,.2f}'.format)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# Plot settings
sns.set_style("whitegrid")
plt.rcParams['figure.dpi'] = 100

print("✓ All imports successful")

✓ All imports successful


---
## 📥 Section 2: Data Loading & Initial Standardization

In [2]:
# ============================================================================
# LOAD RAW DATA
# ============================================================================

df = create_dataframe(DATA_PATH, encoding="latin-1")

print(f"✓ Data loaded successfully from: {DATA_PATH}")
print(f"  Shape: {df.shape[0]} rows × {df.shape[1]} columns")

✓ Data loaded successfully from: /Users/rex/Documents/personal/AquaSafe/data/NWMP_August2025_MPCB_0.csv
  Shape: 222 rows × 54 columns


In [3]:
# ============================================================================
# STANDARDIZE COLUMN NAMES
# ============================================================================
# Normalize to snake_case for consistency with Python conventions

df.columns = (
    df.columns
    .str.strip()
    .str.lower()
    .str.replace(" ", "_")
    .str.replace("/", "_")
    .str.replace("-", "_")
)

print(f"✓ Columns standardized: {df.shape[1]} features normalized to snake_case")

✓ Columns standardized: 54 features normalized to snake_case


In [4]:
# Keep a backup of raw data for comparison
df_raw = df.copy()
print(f"✓ Raw data backup created: {df_raw.shape}")

✓ Raw data backup created: (222, 54)


In [5]:
# Initial missing values check
missing_counts = df.isna().sum()
missing_cols = missing_counts[missing_counts > 0].sort_values(ascending=False)

print(f"\n📊 Initial Missing Values ({len(missing_cols)} columns with NaN):")
print(missing_cols)


📊 Initial Missing Values (44 columns with NaN):
use_of_water_in_down_stream      222
remark                           215
odor                             170
visibility_effluent_discharge     94
major_polluting_sources           73
fecal_streptococci                48
boron                             34
temperature                       29
river_basin                       24
name_of_water_body                22
flouride                          18
magnesium_caco3                    7
total_kjeldahl_n                   7
amonia_n                           7
hardness_caco3                     7
calcium_caco3                      7
total_fixed_solids                 7
sulphate                           7
sodium                             7
total_dissolved_solids             7
chlorides                          7
total_suspended_solids             7
phosphate                          7
potassium                          7
cod                                7
turbidity                 

---
## 🔬 Section 3: Type Normalization (BDL Parsing)

In [6]:
# ============================================================================
# NUMERIC COLUMNS WITH BDL ANNOTATIONS
# ============================================================================
# Problem: Chemical/biological parameters contain "(BDL)" annotations
#          meaning "Below Detection Limit" - lab equipment threshold
# Solution: Extract numeric component + preserve BDL flag as binary feature

NUMERIC_STRING_COLS = [
    "fecal_coliform",
    "total_coliform",
    "fecal_streptococci",
    "total_kjeldahl_n",
    "nitrate_n",
    "turbidity",
    "sulphate",
    "sodium",
    "chlorides",
    "phosphate",
    "boron",
    "potassium",
    "flouride",
    "dissolved_o2",
    "total_suspended_solids",
    "phenophelene_alkanity",
    "total_alkalinity",
]

print(f"✓ Identified {len(NUMERIC_STRING_COLS)} numeric columns with potential BDL annotations")

✓ Identified 17 numeric columns with potential BDL annotations


In [7]:
def parse_numeric_with_bdl(series: pd.Series) -> tuple:
    """
    Convert numeric strings with BDL annotations to float while
    preserving detection-limit information.

    Args:
        series: Column with mixed numeric/annotated values

    Returns:
        tuple: (numeric_values, is_bdl_flag)
    """
    # Identify BDL presence before altering values
    is_bdl = series.astype(str).str.contains("BDL", na=False)

    # Remove annotation and extract numeric portion
    numeric = (
        series.astype(str)
        .str.replace("(BDL)", "", regex=False)
        .str.strip()
    )

    # Coerce to numeric (invalid → NaN)
    numeric = pd.to_numeric(numeric, errors="coerce")

    return numeric, is_bdl

In [8]:
# ============================================================================
# APPLY BDL PARSING
# ============================================================================

conversions_made = 0

for col in NUMERIC_STRING_COLS:
    if col not in df.columns:
        print(f"  ⚠ Column not found: {col}")
        continue

    numeric_values, bdl_flag = parse_numeric_with_bdl(df[col])
    na_count = numeric_values.isna().sum()
    bdl_count = bdl_flag.sum()

    # Replace original column with numeric representation
    df[col] = numeric_values
    conversions_made += 1

    # Preserve detection-limit information as binary flag
    df[f"{col}_is_bdl"] = bdl_flag

    print(f"  ✓ {col}: {bdl_count} BDL flags, {na_count} NaN values")

print(f"\n✓ Numeric normalization complete: {conversions_made} columns processed")

  ✓ fecal_coliform: 28 BDL flags, 7 NaN values
  ✓ total_coliform: 1 BDL flags, 7 NaN values
  ✓ fecal_streptococci: 143 BDL flags, 48 NaN values
  ✓ total_kjeldahl_n: 101 BDL flags, 7 NaN values
  ✓ nitrate_n: 20 BDL flags, 9 NaN values
  ✓ turbidity: 77 BDL flags, 7 NaN values
  ✓ sulphate: 22 BDL flags, 7 NaN values
  ✓ sodium: 22 BDL flags, 7 NaN values
  ✓ chlorides: 2 BDL flags, 7 NaN values
  ✓ phosphate: 101 BDL flags, 11 NaN values
  ✓ boron: 131 BDL flags, 34 NaN values
  ✓ potassium: 91 BDL flags, 7 NaN values
  ✓ flouride: 115 BDL flags, 18 NaN values
  ✓ dissolved_o2: 8 BDL flags, 7 NaN values
  ✓ total_suspended_solids: 40 BDL flags, 7 NaN values
  ✓ phenophelene_alkanity: 168 BDL flags, 10 NaN values
  ✓ total_alkalinity: 5 BDL flags, 7 NaN values

✓ Numeric normalization complete: 17 columns processed


---
## 🌍 Section 4: Geographic Coordinate Standardization

In [9]:
def parse_dms_coordinate(value) -> float:
    """
    Convert geographic coordinates from Degree-Minute format to decimal degrees.

    Input Format: "19°29.263'"
    Output: 19.487716...
    """
    if pd.isna(value):
        return np.nan

    try:
        value = str(value).replace("\ufffd", "°")
        degree_part, minute_part = value.split("°")

        degrees = float(degree_part.strip())
        minutes = float(minute_part.replace("'", "").strip())

        return degrees + (minutes / 60)

    except Exception:
        return np.nan

In [10]:
# ============================================================================
# APPLY COORDINATE CONVERSION
# ============================================================================

df["latitude"] = df["latitude"].apply(parse_dms_coordinate)
df["longitude"] = df["longitude"].apply(parse_dms_coordinate)

print("✓ Coordinates standardized to decimal degrees")
print(f"  Latitude range: [{df['latitude'].min():.2f}, {df['latitude'].max():.2f}]")
print(f"  Longitude range: [{df['longitude'].min():.2f}, {df['longitude'].max():.2f}]")

✓ Coordinates standardized to decimal degrees
  Latitude range: [16.69, 21.27]
  Longitude range: [73.18, 79.20]


---
## 🎯 Section 5: Target Variable Processing

In [11]:
# ============================================================================
# TARGET VARIABLE STANDARDIZATION
# ============================================================================

TARGET_COL = "use_based_class"

# Clean string representation
df[TARGET_COL] = (
    df[TARGET_COL]
    .astype(str)
    .str.strip()
    .replace("nan", np.nan)
)

print(f"✓ Target variable cleaned")
print(f"  Unique values: {df[TARGET_COL].nunique()}")
print(f"\nValue counts:")
print(df[TARGET_COL].value_counts(dropna=False))

✓ Target variable cleaned
  Unique values: 5

Value counts:
use_based_class
A (Drinking Water source without conventional treatment but after disinfection)    141
No Information                                                                      44
E (Irrigation, industrial cooling and controlled waste)                             19
NaN                                                                                  7
C (Drinking water source)                                                            6
B (Outdoor bathing(Organized))                                                       5
Name: count, dtype: int64


In [12]:
# ============================================================================
# TARGET CLASS MAPPING
# ============================================================================
# Map verbose regulatory descriptions to compact codes

TARGET_MAP = {
    "A (Drinking Water source without conventional treatment but after disinfection)": "A",
    "B (Outdoor bathing(Organized))": "B",
    "C (Drinking water source)": "C",
    "E (Irrigation, industrial cooling and controlled waste)": "E",
    "No Information": np.nan,  # Will be removed
}

df[TARGET_COL] = df[TARGET_COL].replace(TARGET_MAP)

print(f"✓ Target mapping applied")
print(f"\nValue counts after mapping:")
print(df[TARGET_COL].value_counts(dropna=False))

✓ Target mapping applied

Value counts after mapping:
use_based_class
A      141
NaN     51
E       19
C        6
B        5
Name: count, dtype: int64


In [13]:
# ============================================================================
# REMOVE ROWS WITH INVALID TARGET
# ============================================================================

rows_before = len(df)
df = df.dropna(subset=[TARGET_COL]).reset_index(drop=True)
rows_after = len(df)

print(f"✓ Removed rows with invalid target")
print(f"  Before: {rows_before} rows")
print(f"  After: {rows_after} rows")
print(f"  Removed: {rows_before - rows_after} rows")
print(f"\nFinal target distribution:")
print(df[TARGET_COL].value_counts().sort_index())

✓ Removed rows with invalid target
  Before: 222 rows
  After: 171 rows
  Removed: 51 rows

Final target distribution:
use_based_class
A    141
B      5
C      6
E     19
Name: count, dtype: int64


---
## 🚫 Section 6: Column Removal (Leakage, Metadata, Identifiers)

In [14]:
# ============================================================================
# COLUMNS TO REMOVE
# ============================================================================

COLUMNS_TO_DROP = [
    # ── Structural Issues (High Missing) ──
    "use_of_water_in_down_stream",  # ~77% missing
    "remark",                        # ~77% missing, free text

    # ── Metadata (Not Water Quality Features) ──
    "sampling_date",
    "sampling_time",
    "month",
    "state_name",
    "mon_agency",
    "frequency",

    # ── Data Leakage Risk ──
    "major_polluting_sources",       # Near-deterministic with target
    "visibility_effluent_discharge", # Highly correlated with classification

    # ── Identifiers (No Predictive Signal) ──
    "stn_code",
    "stn_name",
    "name_of_water_body",
    "district",
    "river_basin",

    # ── Geographic (Exclude from modeling, keep for analysis) ──
    "latitude",
    "longitude",
]

print(f"Columns marked for removal: {len(COLUMNS_TO_DROP)}")
for col in COLUMNS_TO_DROP:
    print(f"  • {col}")

Columns marked for removal: 17
  • use_of_water_in_down_stream
  • remark
  • sampling_date
  • sampling_time
  • month
  • state_name
  • mon_agency
  • frequency
  • major_polluting_sources
  • visibility_effluent_discharge
  • stn_code
  • stn_name
  • name_of_water_body
  • district
  • river_basin
  • latitude
  • longitude


In [15]:
# ============================================================================
# DROP COLUMNS
# ============================================================================

cols_before = df.shape[1]
df = df.drop(columns=COLUMNS_TO_DROP, errors="ignore")
cols_after = df.shape[1]

print(f"\n✓ Column removal complete")
print(f"  Before: {cols_before} columns")
print(f"  After: {cols_after} columns")
print(f"  Removed: {cols_before - cols_after} columns")


✓ Column removal complete
  Before: 71 columns
  After: 54 columns
  Removed: 17 columns


---
## 📊 Section 7: Data Quality Check (Pre-Export)

In [16]:
# ============================================================================
# REMAINING MISSING VALUES
# ============================================================================
# Note: These will be imputed in Notebook 03 AFTER train-test split

missing_counts = df.isna().sum()
missing_cols = missing_counts[missing_counts > 0].sort_values(ascending=False)

print(f"📊 Missing Values Summary (will be imputed after split):")
print(f"   Columns with NaN: {len(missing_cols)}")
print(f"   Total NaN cells: {missing_counts.sum()}")

if len(missing_cols) > 0:
    print(f"\nColumns with missing values:")
    for col, count in missing_cols.items():
        pct = count / len(df) * 100
        print(f"   {col}: {count} ({pct:.1f}%)")

📊 Missing Values Summary (will be imputed after split):
   Columns with NaN: 8
   Total NaN cells: 223

Columns with missing values:
   odor: 133 (77.8%)
   fecal_streptococci: 30 (17.5%)
   temperature: 22 (12.9%)
   boron: 18 (10.5%)
   flouride: 11 (6.4%)
   phosphate: 4 (2.3%)
   phenophelene_alkanity: 3 (1.8%)
   nitrate_n: 2 (1.2%)


In [17]:
# ============================================================================
# COLUMN TYPE SUMMARY
# ============================================================================

numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = df.select_dtypes(include=["object", "string"]).columns.tolist()

print(f"\n📊 Column Type Summary:")
print(f"   Numeric columns: {len(numeric_cols)}")
print(f"   Categorical columns: {len(categorical_cols)}")
print(f"   Total columns: {df.shape[1]}")

print(f"\nCategorical columns:")
for col in categorical_cols:
    print(f"   • {col}: {df[col].nunique()} unique values")


📊 Column Type Summary:
   Numeric columns: 29
   Categorical columns: 8
   Total columns: 54

Categorical columns:
   • type_water_body: 7 unique values
   • use_based_class: 4 unique values
   • weather: 3 unique values
   • approx_depth: 3 unique values
   • human_activities: 18 unique values
   • floating_matter: 2 unique values
   • color: 9 unique values
   • odor: 5 unique values


In [18]:
# ============================================================================
# VALIDATION CHECKS
# ============================================================================

print("\n🔍 Validation Checks:")

# Check 1: Dataset not empty
assert df.shape[0] > 0, "Dataset is empty"
print(f"   ✓ Dataset has {df.shape[0]} rows")

# Check 2: Target column exists and is complete
assert TARGET_COL in df.columns, "Target column missing"
assert df[TARGET_COL].isna().sum() == 0, "Target has missing values"
print(f"   ✓ Target column complete (no NaN)")

# Check 3: Valid target classes
valid_classes = {"A", "B", "C", "E"}
actual_classes = set(df[TARGET_COL].unique())
assert actual_classes.issubset(valid_classes), f"Invalid classes: {actual_classes - valid_classes}"
print(f"   ✓ Target classes valid: {sorted(actual_classes)}")

# Check 4: No duplicate rows
dup_count = df.duplicated().sum()
print(f"   ✓ Duplicate rows: {dup_count}")

print("\n✅ All validation checks passed")


🔍 Validation Checks:
   ✓ Dataset has 171 rows
   ✓ Target column complete (no NaN)
   ✓ Target classes valid: ['A', 'B', 'C', 'E']
   ✓ Duplicate rows: 0

✅ All validation checks passed


---
## 💾 Section 8: Export Cleaned Data

In [19]:
# ============================================================================
# CREATE OUTPUT DIRECTORIES
# ============================================================================

csv_folder = os.path.join(DATA_DIR, "processed", "csv")
parquet_folder = os.path.join(DATA_DIR, "processed", "parquet")

Path(csv_folder).mkdir(parents=True, exist_ok=True)
Path(parquet_folder).mkdir(parents=True, exist_ok=True)

print(f"✓ Output directories ready")

✓ Output directories ready


In [20]:
# ============================================================================
# EXPORT CLEANED DATA
# ============================================================================
# Note: This file contains NaN values - imputation happens after split

OUTPUT_CSV = os.path.join(csv_folder, "cleaned_water_quality.csv")
OUTPUT_PARQUET = os.path.join(parquet_folder, "cleaned_water_quality.parquet")

# Export CSV
df.to_csv(OUTPUT_CSV, index=False)
print(f"✓ Exported: {OUTPUT_CSV}")

# Export Parquet
df.to_parquet(OUTPUT_PARQUET, index=False)
print(f"✓ Exported: {OUTPUT_PARQUET}")

print(f"\n📊 Export Summary:")
print(f"   Rows: {df.shape[0]}")
print(f"   Columns: {df.shape[1]}")
print(f"   File size (CSV): {os.path.getsize(OUTPUT_CSV) / 1024:.1f} KB")

✓ Exported: /Users/rex/Documents/personal/AquaSafe/data/processed/csv/cleaned_water_quality.csv
✓ Exported: /Users/rex/Documents/personal/AquaSafe/data/processed/parquet/cleaned_water_quality.parquet

📊 Export Summary:
   Rows: 171
   Columns: 54
   File size (CSV): 50.0 KB


---
## 📋 Section 9: Cleaning Summary

In [21]:
# ============================================================================
# CLEANING TRANSFORMATION SUMMARY
# ============================================================================

print("\n" + "="*80)
print("📋 DATA CLEANING SUMMARY")
print("="*80)

print(f"""
TRANSFORMATIONS APPLIED:
------------------------
1. Column names standardized to snake_case
2. BDL annotations parsed: {len(NUMERIC_STRING_COLS)} columns → numeric + binary flags
3. Coordinates converted: DMS → Decimal Degrees
4. Target mapped: Verbose labels → A/B/C/E codes
5. Invalid target rows removed: {rows_before - rows_after} rows
6. Problematic columns dropped: {len(COLUMNS_TO_DROP)} columns

OUTPUT DATASET:
---------------
• Rows: {df.shape[0]}
• Columns: {df.shape[1]}
• Numeric features: {len(numeric_cols)}
• Categorical features: {len(categorical_cols)}
• Missing values: {df.isna().sum().sum()} (will be imputed after split)

IMPORTANT NOTES:
----------------
⚠️ This file contains NaN values intentionally
⚠️ Imputation will happen in Notebook 03 AFTER train-test split
⚠️ This prevents data leakage from test set to training set
""")

print("="*80)
print("✅ Data cleaning complete - Ready for feature engineering")
print("="*80)


📋 DATA CLEANING SUMMARY

TRANSFORMATIONS APPLIED:
------------------------
1. Column names standardized to snake_case
2. BDL annotations parsed: 17 columns → numeric + binary flags
3. Coordinates converted: DMS → Decimal Degrees
4. Target mapped: Verbose labels → A/B/C/E codes
5. Invalid target rows removed: 51 rows
6. Problematic columns dropped: 17 columns

OUTPUT DATASET:
---------------
• Rows: 171
• Columns: 54
• Numeric features: 29
• Categorical features: 8
• Missing values: 223 (will be imputed after split)

IMPORTANT NOTES:
----------------
⚠️ This file contains NaN values intentionally
⚠️ Imputation will happen in Notebook 03 AFTER train-test split
⚠️ This prevents data leakage from test set to training set

✅ Data cleaning complete - Ready for feature engineering
